In [225]:
from gurobipy import Model, quicksum, GRB

In [226]:
Plnt = {'plnt'}
DC = set(range(1,3))
Cust = DC.copy()
TRoute = {(i,j) for i in Plnt for j in DC}
DRoute = {(j,k) for j in DC for k in Cust}
Route = T_Route | D_Route
CT = {1,2}

T = 2
Period = list(range(1, T+1))

In [227]:
tp_cost = {(i,j):5 for i,j in TRoute}
del_cost = {(j, k):10 for j,k in DRoute}
demand = {(k,t):10 for k in Cust for t in Period}
dc_run_cost = {(j,ct):1000 for j in DC for ct in CT}
dc_inv_cost = {j:5 for j in DC}

In [228]:
model = Model()

In [229]:
x, y, dc_inv, dc_minus_inv = {}, {}, {}, {}

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for j in DC:
    for ct in CT:
        for t in Period:
            y[j,ct,t] = model.addVar(vtype="B", name=f'y[{j},{ct},{t}]')
    
for j in DC:
    for t in Period:
        dc_inv[j,t] = model.addVar(vtype="C", name=f'dc_inv[{j},{t}]')
    
for j in DC:
    dc_minus_inv[j] = model.addVar(vtype="C", name=f'dc_minus_inv[{j}]')

model.update()

In [234]:
Cust_Demand_Cons, DC_Running_Cons, DC_Flow_Cons = {}, {}, {}

for k in Cust:
    for t in Period:
        Cust_Demand_Cons[k,t] = model.addConstr(
            quicksum(x[j,k,t] for j in DC)
            ==
            demand[k,t]
        )

for j,k in DRoute:
    for t in Period:
        DC_Running_Cons[j,k,t] = model.addConstr(
            x[j,k,t]
            <=
            demand[k,t] * quicksum(y[j,ct,t] for ct in CT) 
        )

for j in DC:
    for t in Period:
        DC_Flow_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt)
            + (0 if t==1 else dc_inv[j,t-1] )
            ==
            quicksum(x[j,k,t] for k in Cust)
            + dc_inv[j,t]
        )

model.addConstr(dc_inv[1,1] == 30)
    
model.update()

In [235]:
model.setObjective(
    quicksum(tp_cost[i,j] * x[i,j,t] for i,j in TRoute for t in Period) +
    quicksum(del_cost[j,k] * x[j,k,t] for j,k in DRoute for t in Period) +
    quicksum(dc_run_cost[j,ct] * y[j,ct,t] for j in DC for ct in CT for t in Period) +
    quicksum(dc_inv_cost[j] * dc_inv[j,t] for j in DC for t in Period)
    ,GRB.MINIMIZE
)
model.update()

In [236]:
model.optimize()

Optimize a model with 34 rows, 36 columns and 102 nonzeros
Variable types: 28 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 3e+01]

MIP start did not produce a new incumbent solution

Presolve removed 0 rows and 12 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [237]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

AttributeError: b"Unable to retrieve attribute 'X'"